In [1]:
# Step1: Import data
import pandas as pd

# Read data
train = pd.read_csv("labeledTrainData.tsv", header = 0, delimiter = "\t", quoting = 3)
test = pd.read_csv("testData.tsv", header = 0, delimiter = "\t", quoting = 3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header = 0, delimiter = "\t", quoting = 3)

# See the number of reviews
print("In train review = %d, in test review = %d, in unlabeled_train review = %d" \
      % (train["review"].size, test["review"].size, unlabeled_train["review"].size))


In train review = 25000, in test review = 25000, in unlabeled_train review = 50000


In [5]:
# Step2: Create a "review_to_wordlist" for cleaning paragraphs/sentences into words
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review, "lxml").get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [6]:
# Step3: Create our main method "review_to_sentences" which contains "review_to_wordlist"
# i.e. convert paragraphs into sentences -> "review_to_sentences"
# split sentences to words -> "review_to_wordlist"

import nltk.data
nltk.download()

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review, tokenizer, remove_stopwords = False):
    # 1. Split paragraphs to sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    for i in raw_sentences:
        if len(i) > 0:
            sentences.append(review_to_wordlist(i, remove_stopwords))
    
    # 2. return to sentences
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [7]:
# Step4: Apply our method "review_to_sentences" to data
sentences = []
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

### Warning signs make the sentences shrink to around 12000

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 15: ordinal not in range(128)

In [33]:
print len(sentences)
print sentences[4]

12950
[u'some', u'may', u'call', u'mj', u'an', u'egotist', u'for', u'consenting', u'to', u'the', u'making', u'of', u'this', u'movie', u'but', u'mj', u'and', u'most', u'of', u'his', u'fans', u'would', u'say', u'that', u'he', u'made', u'it', u'for', u'the', u'fans', u'which', u'if', u'true', u'is', u'really', u'nice', u'of', u'him', u'the', u'actual', u'feature', u'film', u'bit', u'when', u'it', u'finally', u'starts', u'is', u'only', u'on', u'for', u'minutes', u'or', u'so', u'excluding', u'the', u'smooth', u'criminal', u'sequence', u'and', u'joe', u'pesci', u'is', u'convincing', u'as', u'a', u'psychopathic', u'all', u'powerful', u'drug', u'lord']


In [34]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-04-02 17:14:58,259 : INFO : collecting all words and their counts
2018-04-02 17:14:58,262 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-02 17:14:58,438 : INFO : PROGRESS: at sentence #10000, processed 272497 words, keeping 985 word types


Training model...


2018-04-02 17:14:58,486 : INFO : collected 985 word types from a corpus of 352848 raw words and 12950 sentences
2018-04-02 17:14:58,490 : INFO : Loading a fresh vocabulary
2018-04-02 17:14:58,498 : INFO : min_count=40 retains 667 unique words (67% of original 985, drops 318)
2018-04-02 17:14:58,501 : INFO : min_count=40 leaves 346798 word corpus (98% of original 352848, drops 6050)
2018-04-02 17:14:58,510 : INFO : deleting the raw counts dictionary of 985 items
2018-04-02 17:14:58,517 : INFO : sample=0.001 downsamples 77 most-common words
2018-04-02 17:14:58,520 : INFO : downsampling leaves estimated 243578 word corpus (70.2% of prior 346798)
2018-04-02 17:14:58,530 : INFO : estimated required memory for 667 words and 300 dimensions: 1934300 bytes
2018-04-02 17:14:58,534 : INFO : resetting layer weights
2018-04-02 17:14:58,558 : INFO : training model with 4 workers on 667 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2018-04-02 17:14:58,982 : INFO : wor

In [27]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier

from KaggleWord2VecUtility import KaggleWord2VecUtility


# ****** Define functions to create average word vectors
#

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    #
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    #
    # Initialize a counter
    counter = 0.
    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    #
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       #
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs


def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews["review"]:
        clean_reviews.append( KaggleWord2VecUtility.review_to_wordlist( review, remove_stopwords=True ))
    return clean_reviews



if __name__ == '__main__':

    # Read data from files
    train = pd.read_csv( os.path.join(os.path.dirname(__file__), 'data', 'labeledTrainData.tsv'), header=0, delimiter="\t", quoting=3 )
    test = pd.read_csv(os.path.join(os.path.dirname(__file__), 'data', 'testData.tsv'), header=0, delimiter="\t", quoting=3 )
    unlabeled_train = pd.read_csv( os.path.join(os.path.dirname(__file__), 'data', "unlabeledTrainData.tsv"), header=0,  delimiter="\t", quoting=3 )

    # Verify the number of reviews that were read (100,000 in total)
    print "Read %d labeled train reviews, %d labeled test reviews, " \
     "and %d unlabeled reviews\n" % (train["review"].size,
     test["review"].size, unlabeled_train["review"].size )



    # Load the punkt tokenizer
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


    import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append(review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences

    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences

    print "Parsing sentences from training set"
    for review in train["review"]:
        sentences += review_to_sentences(review, tokenizer)

    print "Parsing sentences from unlabeled set"
    for review in unlabeled_train["review"]:
        sentences += review_to_sentences(review, tokenizer)

ImportError: No module named KaggleWord2VecUtility